DropOut

1) we will apply dropout in the hiddenlayers

2) Applied after relu activation function

3) Randomly turnoff p% of neuron in hiddenlayer uring each forward pass.
   Here we are simplyfing the model aechitecture.

4) This has a regularization effect -> reduces overfitting

5) during evaluation dropout are not used

nn.DropOut(p=0.8)



Batch Normalization

1) Applied to Hiddenlayers

2) Applied after linear layer before activation function

3) Normalize activations -> By taking the mean and variance

4) Includes Learnable parameters -> Gamma and beta

5) Improves Training stability -> Reduces the covariance shift

6) Regularization effect

7) During evaluation we will not calculate the BN ... what we calculate at the trainging time that will be the value of that.

nn.BatchNorm1d(layers nurons)

Regularization

1) Applied to Model weights ....not on model bias

2) Introduce via loss function (Penalty term)

3)  w<- w-n*(Loss+penalty) <- weight decay

4) Penalize large weights

5) controlled by hyper parameter (lamdha)

optimizer = torch.optim.SGD(model.parameeters(),learning rate,weight_decay=1e-4->(10^-4))

In [ ]:
import pandas as pd

data = pd.read_csv("/content/fmnist_small.csv")

data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0.0,7.0,0.0,50.0,205.0,196.0,213.0,165.0,0.0,0.0
1,7,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,1,0,0,0,...,142.0,142.0,142.0,21.0,0.0,3.0,0.0,0.0,0.0,0.0
3,8,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0,0,0,0,0,0,0,0,0,...,213.0,203.0,174.0,151.0,188.0,10.0,0.0,0.0,0.0,0.0


In [ ]:
import torch
from sklearn.model_selection import train_test_split

X = data.iloc[:,1:].values
y=data.iloc[:,0].values


In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=143)


In [ ]:
# scaleing
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
from torch.utils.data import Dataset,DataLoader
class Custom_dataset(Dataset):

  def __init__(self,feature,label):
    self.feature = torch.tensor(feature,dtype=torch.float32)
    self.label = torch.tensor(label,dtype=torch.long)

  def __len__(self):
    return len(self.feature)

  def __getitem__(self,index):
    return self.feature[index],self.label[index]



In [ ]:
train_data = Custom_dataset(X_train,y_train)
test_data = Custom_dataset(X_test,y_test)

In [ ]:
train_loader = DataLoader(train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(test_data,batch_size=32,shuffle=False)

In [ ]:
import torch.nn as nn
class My_model(nn.Module):

  def __init__(self,X_feature):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(X_feature.shape[1],128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(64,10)
    )
  def forward(self,X_feature):
    output=self.model(X_feature)
    return output

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = My_model(X_train)
model=model.to(device)

loss_func= nn.CrossEntropyLoss()

epochs =100
learing_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learing_rate,weight_decay=1e-4)

In [ ]:
# Training
for epoch in range(epochs):
  total_loss=0
  for batch_feature,batch_label in train_loader:
    batch_feature,batch_label=batch_feature.to(device),batch_label.to(device)

    y_pred = model(batch_feature)

    loss = loss_func(y_pred,batch_label)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    total_loss+=loss.item()

  avg_loss = total_loss/len(train_loader)

  print(f'{epoch+1}: loss:{avg_loss}')




1: loss:1.8018062584836718
2: loss:1.350883297517266
3: loss:1.1457622974691257
4: loss:1.0051641296332992
5: loss:0.9115705842703161
6: loss:0.8552136244908185
7: loss:0.79686816981141
8: loss:0.7271729984753569
9: loss:0.6955980561988454
10: loss:0.6767529509436916
11: loss:0.6505129509408709
12: loss:0.6272890870000275
13: loss:0.5981384456997186
14: loss:0.5657694234814442
15: loss:0.5529082692425016
16: loss:0.5374276763956312
17: loss:0.5155599184439216
18: loss:0.5226674033722407
19: loss:0.502825184187419
20: loss:0.4875607671032489
21: loss:0.48098917536332575
22: loss:0.4447025715465277
23: loss:0.45299238680114207
24: loss:0.4179533172241399
25: loss:0.44696611942539755
26: loss:0.4117198112145276
27: loss:0.4088040670038949
28: loss:0.3949869075711344
29: loss:0.3912993610744745
30: loss:0.3829187874642896
31: loss:0.3621803415073475
32: loss:0.37001018561947513
33: loss:0.35178179115476743
34: loss:0.34850724943926636
35: loss:0.33862357408228055
36: loss:0.345715608605196

In [ ]:
model.eval()

My_model(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
# evaluation code

total = 0
correct = 0

with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    batch_feature,batch_label=batch_feature.to(device),batch_label.to(device)
    outputs=model(batch_features)

    predicted_labels=torch.max(outputs,1)[1]

    total=total+batch_labels.shape[0]

    correct+=(predicted_labels==batch_labels).sum().item()

print(correct/total)


0.8088495575221238


In [ ]:
# evaluation code

total = 0
correct = 0

with torch.no_grad():
  for batch_features,batch_labels in train_loader:
    batch_feature,batch_label=batch_feature.to(device),batch_label.to(device)
    outputs=model(batch_features)

    predicted_labels=torch.max(outputs,1)[1]

    total=total+batch_labels.shape[0]

    correct+=(predicted_labels==batch_labels).sum().item()

print(correct/total)


0.9738938053097345


From the above we can observe that we can reduce the gap b/w the train and test by drop and regularization and bn.

Hyper parameter tuning using optuna
Bayes's search

Plan of action
1) Number of hidden layers
2) Number of neurons per layer
3) Number of epochs
4) Optimizers
5) Learning Rate
6) Batch Size
7) DropOut rate
8) Weight Delay(Lamdha value)

Objective function

-> Search space
-> model initialization
-> parameter initialization -> like lr,lossfunc
-> training loop
-> Evalution

Study Object
and need to tell the number of trials

In [ ]:
!pip3 install optuna

In [ ]:
import torch.nn as nn
class My_model(nn.Module):

  def __init__(self,input_dim,output_dim,num_hidden_layers,neurons_per_layer):
    super().__init__()

    layers = []

    for i in range(num_hidden_layers):

      layers.append(nn.Linear(input_dim,neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(0.3))
      input_dim=neurons_per_layer
    layers.append(nn.Linear(neurons_per_layer,output_dim))

    self.model = nn.Sequential(*layers) # Here the Seqential will not directly accept the list so we unpack the list by *

  def forward(self,X_feature):
    output=self.model(X_feature)
    return output

In [ ]:
 # Objective function

def Objective_function(trial):

  number_of_layers = trial.suggest_int('number_of_layers',1,5)
  neurons_per_layer = trial.suggest_int('neuron_per_layer',8,128,step=8)

  input_dim=784
  output_dim =10

  model = My_model(input_dim,output_dim,number_of_layers,neurons_per_layer)
  model.to(device)

  learning_rate=0.1
  epochs=100

  loss_func= nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(),lr=learing_rate,weight_decay=1e-4)

  for epoch in range(epochs):
    total_loss=0
    for batch_feature,batch_label in train_loader:
      batch_feature,batch_label=batch_feature.to(device),batch_label.to(device)

      y_pred = model(batch_feature)

      loss = loss_func(y_pred,batch_label)

      optimizer.zero_grad()

      loss.backward()

      optimizer.step()

      total_loss+=loss.item()

    avg_loss = total_loss/len(train_loader)

  # Evaluation
  model.eval()
  total = 0
  correct = 0

  with torch.no_grad():
    for batch_features,batch_labels in test_loader:
      batch_feature,batch_label=batch_feature.to(device),batch_label.to(device)
      outputs=model(batch_features)

      predicted_labels=torch.max(outputs,1)[1]

      total=total+batch_labels.shape[0]

      correct+=(predicted_labels==batch_labels).sum().item()
    accuracy = correct/total
  return accuracy

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(Objective_function,n_trials=10)

[I 2025-07-07 16:27:52,305] A new study created in memory with name: no-name-926b88ca-b742-49e0-8f81-cad5159be4cd
[I 2025-07-07 16:28:17,167] Trial 0 finished with value: 0.8336283185840708 and parameters: {'number_of_layers': 3, 'neuron_per_layer': 112}. Best is trial 0 with value: 0.8336283185840708.
[I 2025-07-07 16:28:34,174] Trial 1 finished with value: 0.7964601769911505 and parameters: {'number_of_layers': 1, 'neuron_per_layer': 128}. Best is trial 0 with value: 0.8336283185840708.
[I 2025-07-07 16:28:58,695] Trial 2 finished with value: 0.8176991150442477 and parameters: {'number_of_layers': 4, 'neuron_per_layer': 48}. Best is trial 0 with value: 0.8336283185840708.
[I 2025-07-07 16:29:21,047] Trial 3 finished with value: 0.7433628318584071 and parameters: {'number_of_layers': 4, 'neuron_per_layer': 16}. Best is trial 0 with value: 0.8336283185840708.
[I 2025-07-07 16:29:46,398] Trial 4 finished with value: 0.8389380530973451 and parameters: {'number_of_layers': 3, 'neuron_per_

In [ ]:
study.best_value

0.8389380530973451

In [ ]:
study.best_params

{'number_of_layers': 3, 'neuron_per_layer': 120}